Youtube links: https://www.youtube.com/watch?v=8nNTG5fH9do&list=PLNRxk1s77zfj2M2MuCuEPy_k25bUbv1Py&index=16

In [0]:
"""
Input:
+----+
|team|
+----+
| PAK|
| AFG|
| IND|
| AUS|
+----+
O/p:
+-----------------+
|Match Combination|
+-----------------+
|AFG Vs PAK       |
|AFG Vs IND       |
|AFG Vs AUS       |
|IND Vs PAK       |
|AUS Vs PAK       |
|AUS Vs IND       |
+-----------------+

select concat(t1.teams, " Vs ", t2.teams) as "Match Combination" from teams t1 join teams t2 on t1.teams<t2.teams
"""


from pyspark.sql.functions import col, concat_ws
# Sample team list
teams = [("PAK",), ("AFG",), ("IND",), ("AUS",)]
df = spark.createDataFrame(teams, ["team"])

df.show()

# Perform self-join
joined_df = df.alias("df1").join(df.alias("df2"),
                                 col("df1.team") < col("df2.team")) 

# Combine into match string
result_df = joined_df.select(concat_ws(" Vs ", col("df1.team"), col("df2.team")).alias("Match Combination"))

# Display result
result_df.show(truncate=False)

In [0]:
"""
Input Table1:
+---+-----+
| Id| Name|
+---+-----+
|  1|Steve|
|  2|David|
|  3| John|
|  4|Shree|
|  5|Helen|
+---+-----+
Input Table2:
+-------+-----+---+
|Subject|Marks| Id|
+-------+-----+---+
|    SQL|   90|  1|
|PySpark|  100|  1|
|    SQL|   70|  2|
|PySpark|   60|  2|
|    SQL|   30|  3|
|PySpark|   20|  3|
|    SQL|   50|  4|
|PySpark|   50|  4|
|    SQL|   45|  5|
|PySpark|   45|  5|
+-------+-----+---+

Output:
+---+-----+----------+------------+
| Id| Name|Percentage|      Result|
+---+-----+----------+------------+
|  1|Steve|      95.0| Distinction|
|  2|David|      65.0|Second Class|
|  3| John|      25.0|        Fail|
|  4|Shree|      50.0|Second Class|
|  5|Helen|      45.0|        Fail|
+---+-----+----------+------------+
SQL:
SELECT 
    s.Id,
    s.Name,
    ROUND(AVG(m.Marks), 1) AS Percentage,
    CASE 
        WHEN AVG(m.Marks) >= 75 THEN 'Distinction'
        WHEN AVG(m.Marks) >= 50 THEN 'Second Class'
        ELSE 'Fail'
    END AS Result
FROM 
    Table1 s
JOIN 
    Table2 m ON s.Id = m.Id
GROUP BY 
    s.Id, s.Name;
"""
from pyspark.sql.functions import col, sum, when, avg

data1 = [(1, "Steve"), (2, "David"), (3, "John"), (4, "Shree"), (5, "Helen")]
data2 = [("SQL", 90, 1), ("PySpark", 100, 1), ("SQL", 70, 2), ("PySpark", 60, 2),
         ("SQL", 30, 3), ("PySpark", 20, 3), ("SQL", 50, 4), ("PySpark", 50, 4),
         ("SQL", 45, 5), ("PySpark", 45, 5)]

schema1 = ["Id", "Name"]
schema2 = ["Subject", "Marks", "Id"]


df1 = spark.createDataFrame(data1, schema1)
df2 = spark.createDataFrame(data2, schema2)

df1.show()
df2.show()



join_marks_df = df1.join(df2, "Id")
percentage_df = join_marks_df.groupBy("Id", "Name").agg((avg(col("Marks"))).alias("Percentage"))
percentage_df.show()

final_df = percentage_df.select("Id", "Name", "Percentage")\
                        .withColumn("Result", 
                                    when(col("Percentage")>70, "Distinction")
                                    .when(col("Percentage")>=50, "Second Class")
                                    .when(col("Percentage")<50, "Fail"))
final_df.show()



In [0]:
"""
Diplay percentage of each product contribution in the store.
Input:
-------------------------------------
Store       |   Product    |    Sales
-------------------------------------
S1          |   P1         |    100
S1          |   P2         |    200
S2          |   P1         |    300
S2          |   P2         |    400
S3          |   P1         |    500
S3          |   P2         |    600
--------------------------------------

Output:
+-----+-------+-----+---------------------+---------------+
|Store|Product|Sales|Total Sales Per Store|Product Sales %|
+-----+-------+-----+---------------------+---------------+
|   S1|     P1|  100|                  300|          33.33|
|   S1|     P2|  200|                  300|          66.67|
|   S2|     P1|  300|                  700|          42.86|
|   S2|     P2|  400|                  700|          57.14|
|   S3|     P1|  500|                 1100|          45.45|
|   S3|     P2|  600|                 1100|          54.55|
+-----+-------+-----+---------------------+---------------+
"""
from pyspark.sql.functions import col, sum, round

data = [("S1","P1",100),("S1","P2",200),("S2","P1",300),("S2","P2",400),("S3","P1",500),("S3","P2",600)]
colums = ["Store","Product","Sales"]

df1 = spark.createDataFrame(data, colums)

#Total sales per store
df_total_sales_per_store = df1.groupBy("Store").agg(sum("Sales").alias("Total Sales Per Store"))

#Join total sales to the original df
joined_df = df1.join(df_total_sales_per_store, "Store")

#caliculating contribution
df_product_sales = joined_df.withColumn("Product Sales %", round(((col("Sales")/col("Total Sales Per Store"))*100),2))

df_product_sales.show()

In [0]:
"""
Listing out month. year from the given input date
+--------+----------+------+
|order_id|order_date| price|
+--------+----------+------+
|    1001|2025-06-03|150.75|
|    1002|2025-06-14| 89.99|
|    1003|2025-06-25|200.00|
|    1004|2025-07-01|175.20|
|    1005|2025-07-11| 49.95|
|    1006|2025-07-20|300.50|
|    1007|2025-08-02|120.00|
|    1008|2025-08-05|245.75|
|    1009|2025-08-12|180.40|
|    1010|2025-08-18| 95.99|
+--------+----------+------+
"""

from pyspark.sql.functions import month, year, date_format

from datetime import datetime

# Sample data
data = [
    (1001, datetime(2025, 6, 3), 150.75),
    (1002, datetime(2025, 6, 14), 89.99),
    (1003, datetime(2025, 6, 25), 200.00),
    (1004, datetime(2025, 7, 1), 175.20),
    (1005, datetime(2025, 7, 11), 49.95),
    (1006, datetime(2025, 7, 20), 300.50),
    (1007, datetime(2025, 8, 2), 120.00),
    (1008, datetime(2025, 8, 5), 245.75),
    (1009, datetime(2025, 8, 12), 180.40),
    (1010, datetime(2025, 8, 18), 95.99)
]

colums = ["order_id", "order_date", "price"]

df = spark.createDataFrame(data, colums)

#fetch only month and year separately in another colums
df = df.withColumn("month", date_format("order_date","MMMM" )).withColumn("year", year("order_date"))
display(df)

In [0]:
"""
Input Table:
----------------------------+
|Name    | Hobbies           |
----------------------------+
|John    | Reading, Writing  |
|Mary    | Painting, Dancing |
|Peter   | Singing, Dancing  |
-----------------------------

O/p Table:
+-----+--------+
| Name| Hobbies|
+-----+--------+
| John| Reading|
| John| Writing|
| Mary|Painting|
| Mary| Dancing|
|Peter| Singing|
|Peter| Dancing|
+-----+--------+

"""
from pyspark.sql.functions import split, explode

# Sample 
data = [
    ("John", "Reading, Writing"),
    ("Mary", "Painting, Dancing"),
    ("Peter", "Singing, Dancing")
]
colums = ["Name", "Hobbies"]

df = spark.createDataFrame(data, colums)

df = df.select("Name", explode(split("Hobbies",",")).alias("Hobbies"))

df.show()


In [0]:
"""
Input:
+-----+--------+
| Name| Hobbies|
+-----+--------+
| John| Reading|
| John| Writing|
| Mary|Painting|
| Mary| Dancing|
|Peter| Singing|
|Peter| Dancing|
+-----+--------+
O/p:
-----------------------------+
|Name    | Hobbies           |
-----------------------------+
|John    | Reading, Writing  |
|Mary    | Painting, Dancing |
|Peter   | Singing, Dancing  |
-----------------------------+
SQL:
SELECT 
    Name, 
    GROUP_CONCAT(Hobbies SEPARATOR ', ') AS Hobbies
FROM input_table
GROUP BY Name;
"""
from pryspark.sql.functions import collect_list, concat_ws
df = df.groupBy("Name").agg(concat_ws(",", collect_list(df.Hobbies).alias("Skills")))
display(df)

In [0]:
"""
Input Table:
+-----+-----+-----+
|city1|city2|city3|
+-----+-----+-----+
|Goa  |AP   |     |
|     |AP   |NULL |
|NULL |     |bglr |
+-----+-----+-----+

O/p Table: (Ignore Empty & Null values. Put all the first not null value in the last column)
+-----+-----+-----+----------------+
|city1|city2|city3|FirstNotNullCity|
+-----+-----+-----+----------------+
|  Goa|   AP|     |             Goa|
|     |   AP| NULL|              AP|
| NULL|     | bglr|            bglr|
+-----+-----+-----+----------------+

select city1, city2, city3, coalesce(NULLIF(city1,""), NULLIF(city2,""), NULLIF(city3,"")) as FirstNotNullCity from Cities
"""
from pyspark.sql.functions import coalesce, when, col
data = [("Goa", "AP", ""), ("", "AP", None),(None, "", "bglr")]
colums = ["city1", "city2", "city3"]
df = spark.createDataFrame(data, colums)
df.show(truncate=False)

result_df = df.withColumn("FirstNotNullCity", coalesce(when(df.city1 == "", None).otherwise(df.city1),
                                                       when(df.city2 == "", None).otherwise(df.city2),
                                                       when(df.city3 == "", None).otherwise (df.city3)))

result_df.show()


In [0]:
"""
Display highest salary from each department
+---+----+------+----------+
|Id |Name|Salary|Department|
+---+----+------+----------+
|1  |A   |1000  |IT        |
|2  |B   |1500  |IT        |
|3  |C   |2500  |IT        |
|4  |D   |3000  |HR        |
|5  |E   |2000  |HR        |
|6  |F   |1000  |HR        |
|7  |G   |4000  |Sales     |
|8  |H   |4000  |Sales     |
|9  |I   |1000  |Sales     |
|10 |J   |2000  |Sales     |
+---+----+------+----------+

Output:
+---+----+------+----------+----+
| Id|Name|Salary|Department|rank|
+---+----+------+----------+----+
|  4|   D|  3000|        HR|   1|
|  3|   C|  2500|        IT|   1|
|  7|   G|  4000|     Sales|   1|
|  8|   H|  4000|     Sales|   1|
+---+----+------+----------+----+

SQL:
select id, name, salary, department, rank() over(partition by department order by salary desc) as rank from employee qualify rank=1
"""

from pyspark.sql.functions import dense_rank, col
from pyspark.sql.window import Window

data = [
    (1, "A", 1000, "IT"),
    (2, "B", 1500, "IT"),
    (3, "C", 2500, "IT"),
    (4, "D", 3000, "HR"),
    (5, "E", 2000, "HR"),
    (6, "F", 1000, "HR"),
    (7, "G", 4000, "Sales"),
    (8, "H", 4000, "Sales"),
    (9, "I", 1000, "Sales"),
    (10, "J", 2000, "Sales")
]
colums = ["Id", "Name", "Salary", "Department"]

df = spark.createDataFrame(data, colums)

rank_df = df.select("*", dense_rank().over(Window.partitionBy("Department").orderBy(col("Salary").desc())).alias("rank"))

# display highest rank department wise
rank_df= rank_df.filter(rank_df.rank == 1).distinct()
rank_df.show()

In [0]:
"""
Find the age of the person with the given dob column in the data frame.
"""

from pyspark.sql.functions import to_date, current_date, floor, months_between, col

df = spark.createDataFrame([("Gopikrishna", "Bangalore", "1995-05-27")], ["name", "city", "dob"])

# Step 1: Convert 'dob' string to DateType
df = df.withColumn("dob", to_date(col("dob"), "yyyy-MM-dd"))

# Step 2: Calculate age
df_with_age = df.withColumn("age", floor(months_between(current_date(), col("dob")) / 12))

# Optional: show result
df_with_age.select("dob", "age").show()


In [0]:
"""
Find the products that have never been ordered.
Dataset: Products (product_id, product_name), Order_Items (order_id, product_id, quantity)

SELECT product_id, product_name FROM Products WHERE product_id NOT IN (SELECT product_id FROM Order_Items);
"""
# Sample DataFrames
products_df = spark.createDataFrame([(1, "Laptop"),(2, "Mouse"),(3, "Keyboard")], ["product_id", "product_name"])

order_items_df = spark.createDataFrame([(101, 1, 2), (102, 2, 5)], ["order_id", "product_id", "quantity"])

# LEFT ANTI JOIN → keeps rows from left that have no match in right
never_ordered_df = products_df.join(order_items_df, on="product_id", how="left_anti")

never_ordered_df.show()

In [0]:
from pyspark.sql.functions import spark_partition_id

file_path = "/Volumes/workspace/default/volume/employee_data.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

# Original partition count
df.withColumn("PartitionId", spark_partition_id()) \
  .groupBy("PartitionId") \
  .count() \
  .orderBy("PartitionId") \
  .show()

# Repartition to 10
new_df = df.repartition(10)

# New partition count
new_df.withColumn("PartitionId", spark_partition_id()) \
      .groupBy("PartitionId") \
      .count() \
      .orderBy("PartitionId") \
      .show()


"""
from pyspark.sql.functions import spark_partition_id

file_path = "/Volumes/workspace/default/volume/employee_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

df.rdd.getNumPartitions()
df.repartition(10)

df.rdd.getNumPartitions()
df.withColumn("PartitionId", spark_partition_id()) \
  .groupBy("PartitionId") \
  .count() \
  .orderBy("PartitionId") \
  .show()

"""

In [0]:
"""
Task: Identify products whose total sales revenue has increased every year, including product_id, product_name, and category.

Input: sales
+----------+----+-------------------+
|product_id|year|total_sales_revenue|
+----------+----+-------------------+
|         1|2021|                100|
|         1|2022|                150|
|         1|2023|                200|
|         2|2021|                300|
|         2|2022|                250|
|         2|2023|                400|
|         3|2021|                 50|
|         3|2022|                 50|
|         3|2023|                 60|
+----------+----+-------------------+
Input: products
+----------+------------+----------+
|product_id|product_name|  category|
+----------+------------+----------+
|         1|   Product A|Category X|
|         2|   Product B|Category Y|
|         3|   Product C|Category Z|
+----------+------------+----------+

Output: Identify products whose total sales revenue has increased every year, including product_id, product_name, and category.
+----------+------------+----------+
|product_id|product_name|  category|
+----------+------------+----------+
|         1|   Product A|Category X|
+----------+------------+----------+

Youtube URL: https://www.youtube.com/watch?v=aHRy8mTzxNs&ab_channel=DEwithDhairy
"""

from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window

# Sample DataFrames (replace with actual tables)
sales_df = spark.createDataFrame([
 (1, 2019, 1000.00),
 (1, 2020, 1200.00),
 (1, 2021, 1100.00),
 (2, 2019, 500.00),
 (2, 2020, 600.00),
 (2, 2021, 900.00),
 (3, 2019, 300.00),
 (3, 2020, 450.00),
 (3, 2021, 400.00)
], ["product_id", "year", "total_sales_revenue"])

product_df = spark.createDataFrame([
 (1, 'Laptops', 'Electronics'),
 (2, 'Jeans', 'Clothing'),
 (3, 'Chairs', 'Home Appliances')
 ], ["product_id", "product_name", "category"])

# Step 1: Add previous year's revenue
sales_with_lag = sales_df.withColumn("prev_revenue", lag("total_sales_revenue").over(Window.partitionBy("product_id").orderBy("year")))

# Step 2: Flag whether revenue increased
sales_flagged = sales_with_lag.withColumn("increased", (col("total_sales_revenue") - col("prev_revenue")))

# Step 3: Filter out products with any non-increasing year
valid_products = sales_flagged.groupBy("product_id").agg(min(col("increased"))).filter(min(col("increased") > 0))

# Step 4: Join with product details
final_result = valid_products.join(products_df, on="product_id", how="inner") \
    .select("product_id", "product_name", "category")

final_result.show()

In [0]:
"""
subtract - Returns the rows in left DataFrame but not in right DataFrame based on a join of columns. Doesn't include duplicates.
exceptall - Returns the rows in left DataFrame but not in right DataFrame based on a join of columns. Includes duplicates as well.
"""
# Sample DataFrames
df1 = spark.createDataFrame([(1, "A"),(1, "A"),(2, "B"),(3, "C")], ["id", "value"])
df2 = spark.createDataFrame([(1, "A"),(3, "C")], ["id", "value"])

# subtract → removes duplicates from the result
df_subtract = df1.subtract(df2)
print("subtract result (no duplicates):")
df_subtract.show()

# exceptAll → keeps duplicates from the left that don't match in right
df_exceptall = df1.exceptAll(df2)
print("exceptAll result (duplicates kept):")
df_exceptall.show()


In [0]:
"""

PERMISSIVE (default) =>	Corrupt/malformed rows go into a special column _corrupt_record (if columnNameOfCorruptRecord is set), and other columns are set to null.
DROPMALFORMED	     => Entire bad rows are silently dropped.
FAILFAST	         => Job immediately fails if a bad row is found.
"""
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

# Example file path
file_path = "/Volumes/workspace/default/volume/employee_data.csv"

# PERMISSIVE mode (default), columnNameOfCorruptRecord APPEND new column and puts faulty rows in it.
df_permissive = spark.read.csv(file_path, header=True, schema=schema, mode="PERMISSIVE", 
                               columnNameOfCorruptRecord="_corrupt_record")
print("PERMISSIVE mode result:")
df_permissive.show(truncate=False)

# DROPMALFORMED mode, drops the faulty rows
df_drop = spark.read.csv(file_path, header=True, schema=schema, mode="DROPMALFORMED")
print("DROPMALFORMED mode result:")
df_drop.show()

# FAILFAST mode
# This will throw an exception if a bad row is found
df_failfast = spark.read.csv(file_path, header=True, schema=schema, mode="FAILFAST")
df_failfast.show()

In [0]:
"""
lag(column, offset) → gets the value from a previous row within a window.
lead(column, offset) → gets the value from a next row within a window.
"""
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead, col, when

# Example telecom performance data
data = [
    ("SITE_A", "2024-09-01", 4500),
    ("SITE_A", "2024-09-02", 4800),
    ("SITE_A", "2024-09-03", 4300),
    ("SITE_B", "2024-09-01", 5200),
    ("SITE_B", "2024-09-02", 5400),
    ("SITE_B", "2024-09-03", 5600),
]
columns = ["site_id", "date", "traffic_gb"]

df = spark.createDataFrame(data, columns)

# Window partitioned by site and ordered by date
w = Window.partitionBy("site_id").orderBy("date")

# Add previous and next day's traffic
df_trend = (
    df.withColumn("prev_day_traffic", lag("traffic_gb", 1).over(w))
      .withColumn("next_day_traffic", lead("traffic_gb", 1).over(w))
      .withColumn(
          "traffic_change_gb",
          col("traffic_gb") - col("prev_day_traffic")
      )
      .withColumn(
          "spike_flag",
          when(col("traffic_gb") - col("prev_day_traffic") > 300, "SPIKE").otherwise("NORMAL")
      )
)

df_trend.show()


In [0]:
"""
input:
--------
arrayArrayData = [
  ("James",[["Java","Scala","C++"],["Spark","Java"]]),
  ("Michael",[["Spark","Java","C++"],["Spark","Java"]]),
  ("Robert",[["CSharp","VB"],["Spark","Python"]])
]
 
output:
-------
+-------+------------------+
|name   |colName           |
+-------+------------------+
|James  |[Java, Scala, C++]|
|James  |[Spark, Java]     |
|Michael|[Spark, Java, C++]|
|Michael|[Spark, Java]     |
|Robert |[CSharp, VB]      |
|Robert |[Spark, Python]   |
+-------+------------------+
"""
from pyspark.sql.functions import explode, col
# Input data
arrayArrayData = [
    ("James", [["Java", "Scala", "C++"], ["Spark", "Java"]]),
    ("Michael", [["Spark", "Java", "C++"], ["Spark", "Java"]]),
    ("Robert", [["CSharp", "VB"], ["Spark", "Python"]])
]
df = spark.createDataFrame(arrayArrayData, ["name", "courses"])

# Flatten the array of arrays
df_flattened = df.select("name", explode(col("courses")))
display(df_flattened)

In [0]:
"""
Input
-----
col1      col2       col3
a         aa         1
a         aa         2
b         bb         3
b         bb         4
b         bb         5
 
output
-------
col1     col2        col3
a        aa          [1,2]
b        bb          [3,4,5]
"""
from pyspark.sql.functions import collect_list

# Input data
data = [("a", "aa", 1), ("a", "aa", 2), ("b", "bb", 3), ("b", "bb", 4), ("b", "bb", 5)]
schema = ["col1", "col2", "col3"]

df = spark.createDataFrame(data, schema)
df=df.groupBy("col1", "col2").agg(collect_list("col3").alias("col3"))
display(df)

In [0]:
"""
show emp_name, emp_deptid, state where there is same state & same department

-- Employee Table --
+------+--------+-----------+
|emp_id|emp_name|empdept_id |
+------+--------+-----------+
|     1|   Harry|          5|
|     2|    Ron |          5|
|     3| Neville|         10|
|     4|  Malfoy|          5|
+------+--------+-----------+
-- Address Table --
+------+--------------+------+
|emp_id| city         |state |
+------+--------------+------+
|1     | Nagpur       |   MH |
|2     | Pune         |   MH |
|3     | Mumbai       |   MH |
|4     | Chennai      |   TN |
+------+--------------+------+
"""

# Sample data
employee_data = [(1, "Harry", 5),(2, "Ron", 5),(3, "Neville", 10),(4, "Malfoy", 5)]
address_data = [(1, "Nagpur", "MH"),(2, "Pune", "MH"), (3, "Mumbai", "MH"),(4, "Chennai", "TN")]

# Create DataFrames
df_emp = spark.createDataFrame(employee_data, ["emp_id", "emp_name", "empdept_id"])
df_addr = spark.createDataFrame(address_data, ["emp_id", "city", "state"])

# Join
df_joined = df_emp.join(df_addr, "emp_id")

# Find dept_id + state combinations having more than 1 employee
df_filtered_keys = (
    df_joined.groupBy("empdept_id", "state")
             .agg(count("*").alias("cnt"))
             .filter(col("cnt") > 1)
             .select("empdept_id", "state")
)

# Join back to get full details
result_df = (
    df_joined.join(df_filtered_keys, ["empdept_id", "state"])
             .select("emp_name", "empdept_id", "state")
)

result_df.show()

In [0]:
"""
Customers Table:
---------------
Customer_ID Name        City
1           John Levi   New York
2           Jane Tye    Los Angeles
3           Mike Foley  Chicago
4           Alice White New York

Orders Table:
---------------
Order_ID    Customer_ID Order_Date  Order Total
100         1           2023-07-01  100.00
101         2           2023-06-15  50.00
102         3           2023-07-05  150.00
103         1           2023-07-07  75.00
104         4           2023-07-02  200.00
"""

"""
SELECT 
    c.Name,
    COUNT(o.Order_ID) AS total_orders
FROM Customers c
JOIN Orders o 
    ON c.Customer_ID = o.Customer_ID
WHERE MONTH(o.Order_Date) <> 6  -- Exclude June
GROUP BY c.Name;

"""
customers_data = [
    (1, "John Levi", "New York"),
    (2, "Jane Tye", "Los Angeles"),
    (3, "Mike Foley", "Chicago"),
    (4, "Alice White", "New York")
]
orders_data = [
    (100, 1, "2023-07-01", 100.00),
    (101, 2, "2023-06-15", 50.00),
    (102, 3, "2023-07-05", 150.00),
    (103, 1, "2023-07-07", 75.00),
    (104, 4, "2023-07-02", 200.00)
]

from pyspark.sql.functions import month, count
# Create DataFrames
df_customers = spark.createDataFrame(customers_data, ["Customer_ID", "Name", "City"])
df_orders = spark.createDataFrame(orders_data, ["Order_ID", "Customer_ID", "Order_Date", "Order_Total"])

# Filter out June orders, group by customer, count orders
result_df = (
    df_customers.join(df_orders, "Customer_ID")
    .filter(month(col("Order_Date")) != 6)  # Exclude June
    .groupBy("Name")
    .agg(count("Order_ID").alias("total_orders"))
)

result_df.show()

In [0]:
"""
id, age -> Store in a file
name, gender -> Store in another file
marks -> Store in another file
"""
data = [
    (1, "Sagar", 23, "Male", 68.0),
    (2, "Kim", 35, "Female", 90.2),
    (3, "Alex", 40, "Male", 79.1)
]

#schema = "Id int, Name string, Age int, Gender string, Marks float"
schema = ["id", "name", "age", "gender", "marks"]
df = spark.createDataFrame(data, schema)

int_df = df.select("Id", "Age")
int_df.write.mode("overwrite").parquet("path/integers")

str_df = df.select("Name", "Gender")
str_df.write.mode("overwrite").parquet("path/strings")

float_df = df.select("Marks")
float_df.write.mode("overwrite").parquet("path/floats")


"""
from pyspark.sql.functions import col

set_of_dtypes = set([i[1] for i in df.dtypes])

for i in set_of_dtypes:
    cols = []
    for j in df.dtypes:
        if i == j[1]:
            cols.append(j[0])
    df.select(cols).write.mode('overwrite').save(f'/path/{i}')

"""

In [0]:
"""
Inputs:
================================================
Lift Capacity
-----------------
| id | capacity |
| -- | -------- |
| 1  | 300      |
| 2  | 350      |

Passenger
------------------------------------------
| passenger_name  | weigh_kg   | lift_id  |
| --------------- | ---------- | -------- |
| Rahul           | 85         | 1        |
| Adarsh          | 73         | 1        |
| Riti            | 95         | 1        |
| Viraj           | 80         | 1        |
| Vimal           | 83         | 2        |
| Neha            | 77         | 2        |
| Priti           | 73         | 2        |
| Himanshi        | 85         | 2        |

Output:
==================================================
| lift_id  | names                     |
| -------- | ------------------------- |
| 1        | Adarsh,Viraj,Rahul        |
| 2        | Priti,Neha,Vimal,Himanshi |

Task:
Produce a comma-separated list of passengers who can be accommodated in each lift without exceeding the weight capacity of that lift.
"""

from pyspark.sql.window import Window
from pyspark.sql.functions import sum, col, collect_list

# Data
lift_data = [(1, 300), (2, 350)]
passenger_data = [
                    ("Rahul", 85, 1),
                    ("Adarsh", 73, 1),
                    ("Riti", 95, 1),
                    ("Viraj", 80, 1),
                    ("Vimal", 83, 2),
                    ("Neha", 77, 2),
                    ("Priti", 73, 2),
                    ("Himanshi", 85, 2)
                ]

# Schema
lift_schema = "id int, capacity_kg int"
passenger_schema = "passenger_name string, weight_kg int, lift_id int"

# DFs
lift_df = spark.createDataFrame(lift_data, lift_schema)
passenger_df = spark.createDataFrame(passenger_data, passenger_schema)

# Transformations
join_df = lift_df.join(passenger_df, lift_df.id == passenger_df.lift_id, "inner").drop("id")
weights_df = join_df.withColumn("weights", sum(col("weight_kg")).over(Window.partitionBy("lift_id").orderBy("weight_kg")))
capacity_df = weights_df.filter(weights_df.weights <= weights_df.capacity_kg)

final_df = capacity_df.groupBy("lift_id").agg(collect_list("passenger_name").alias("passenger_names"))

display(final_df)

In [0]:
"""
Input: Fill NULL values wih the first non-null value in the column.
+------------+-------------+
| category   | brand name  |
| ---------- | ----------- |
| chocolates | 5-star      |
| NULL       | dairy milk  |
| NULL       | perk        |
| NULL       | eclair      |
| Biscuits   | Britania    |
| NULL       | good day    |
| NULL       | boost       |
+------------+-------------+
Output:
+------------+-------------+
| category   | brand name  |
| ---------- | ----------- |
| chocolates | 5-star      |
| chocolates | dairy milk  |
| chocolates | perk        |
| chocolates | eclair      |
| Biscuits   | Britania    |
| Biscuits   | good day    |
| Biscuits   | boost       |
+------------+-------------+

SELECT 
    LAST_VALUE(category IGNORE NULLS) 
        OVER (ORDER BY brand_name ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS category,
    brand_name
FROM products;

"""

from pyspark.sql.functions import col, last
from pyspark.sql.window import Window

# Data
data = [
    ("chocolates", "5-star"),
    (None, "dairy milk"),
    (None, "perk"),
    (None, "eclair"),
    ("Biscuits", "Britania"),
    (None, "good day"),
    (None, "boost")
]

df = spark.createDataFrame(data, ["category", "brand_name"])

# Define window spec
w = Window.orderBy("brand_name").rowsBetween(Window.unboundedPreceding, 0)

# Forward fill using last() with ignoreNulls=True
df_filled = df.withColumn("category_filled", last("category", True).over(w)) \
              .select(col("category_filled").alias("category"), "brand_name")

df_filled.show()

In [0]:
"""
Input:
+------------------+------------------+--------------------+----------------+
| transaction_type | transaction_date | transaction_amount | transaction_id |
+------------------+------------------+--------------------+----------------+
| credit           | 01-07-25         | 1000               | 1              |
| debit            | 02-07-25         | 200                | 2              |
| credit           | 03-07-25         | 500                | 3              |
+------------------+------------------+--------------------+----------------+
Output:
+------------------+----------------+--------+-------+---------+
| transaction_date | transaction_id | credit | debit | balance |
+------------------+----------------+--------+-------+---------+
| 01-07-25         | 1              | 1000   | NULL  | 1000    |
| 02-07-25         | 2              | NULL   | 200   | 800     |
| 03-07-25         | 3              | 500    | NULL  | 1300    |
+------------------+----------------+--------+-------+---------+

SELECT
  transaction_date,
  transaction_id,
  CASE WHEN transaction_type = 'credit' THEN transaction_amount END AS credit,
  CASE WHEN transaction_type = 'debit' THEN transaction_amount END AS debit,
  SUM(CASE 
        WHEN transaction_type = 'credit' THEN transaction_amount 
        WHEN transaction_type = 'debit' THEN -transaction_amount 
      END) OVER (
        ORDER BY transaction_date, transaction_id
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ) AS balance
FROM transactions;
"""
from pyspark.sql.functions import col, when, lit, sum
from pyspark.sql.window import Window

# Sample data
data = [
    ("credit", "01-07-25", 1000, 1),
    ("debit", "02-07-25", 200, 2),
    ("credit", "03-07-25", 500, 3)
]

df = spark.createDataFrame(data, ["transaction_type", "transaction_date", "transaction_amount", "transaction_id"])

# Add credit and debit columns
df = df.withColumn("credit", when(col("transaction_type") == "credit", col("transaction_amount")).otherwise(lit(0)))\
        .withColumn("debit", when(col("transaction_type") == "debit", col("transaction_amount")).otherwise(lit(0)))

# Define window for running balance
window_spec = Window.orderBy("transaction_date", "transaction_id").rowsBetween(Window.unboundedPreceding, 0)

# Compute running balance
df = df.withColumn("balance", sum(col("credit") - col("debit")).over(window_spec))

# Select final columns
df_final = df.select("transaction_date", "transaction_id", "credit", "debit", "balance")
df_final.show()

In [0]:
"""
Flatten JSON file given: complex json file: flatten this file & save it as a parquet file s3,

"customer": {
    "customerId": "CUST-1001",
    "name": {
      "first": "Aarav",
      "last": "Sharma"
    },
    "email": "aarav.sharma@example.com",
    "phone": "+91-9876543210",
    "address": {
        "line1": "123 MG Road",
        "line2": "Indiranagar",
        "city": "Bengaluru",
        "state": "KA",
        "postalCode": "560038",
        "country": "India"
    }
  }
"""
spark = SparkSession.builder.appName("App").getOrCreate()
df = spark.read.json("s3a://bucket/input.json", multiline=true)
df.printSchema()
flat_df = df.select(col("customer.customerid").alias("customerid)),
			col("customer.name.first").alias("firstName")
)
flat_df.write.mode("overwrite").parquet("s3a://bucket/location/customer_pq")

In [0]:
"""
You have student, score cols in a table. Obtain max score first.

Input:
student, score
Alece 95
John 96
Tom 84
Rechard 87

output:
student, score
John 96
Alece 95
Rechard 87
Tom 84
"""
result = df.orderBy(col("score").desc())

In [0]:
"""
Read the JSON file  {first_name:"XYZ", last_name:"YZX"}
First_name check for NOT_NULL, if it NULL then DROP those records
Derive new column FULL_NAME by concatenating both columns
Display all 3 columns only when the FULL_NAME is greater than 30 characters
"""
df_json = spark.read.json("../Test_Files/file.json")
df_filter = df_json.filter(col("first_name").isNotNull())
df_full_name = df_filter.withColumn("full_name", concat_ws(" ", col("first_name"), col("last_name")))
df_result = df_full_name.filter(length(col("full_name"))>30).\
    select("first_name", "last_name", "full_name").show()

In [0]:
"""
Input:
trips table:
trip_id
client_id	=> foreign key user_id from users
driver_id	=> foreign key user_id from users
status		=> values 'cancelled', 'active', 'completed'
request_dt	

users table:
user_id
banned_status	=> 'yes','no'
role 		=> 'driver','client'

cancellation_rate=total_num_of_rides/cancelled_num_of_rides
PS: only consider users with banned_status='No'
 
Output:
request_dt & cancellation_rate
 
"""
from pyspark.sql.functions import col, when, sum as _sum, count, round

# Non-banned clients
clients = users.filter((col("role") == "client") & (col("banned_status") == "no")) \
               .select(col("user_id").alias("client_id"))

# Non-banned drivers
drivers = users.filter((col("role") == "driver") & (col("banned_status") == "no")) \
               .select(col("user_id").alias("driver_id"))

# Valid trips (only non-banned client & driver)
valid_trips = (trips
               .join(clients, "client_id")
               .join(drivers, "driver_id"))

# Step 1: Add a flag column for cancelled rides
valid_trips = valid_trips.withColumn(
    "is_cancelled",
    when(col("status") == "cancelled", 1).otherwise(0)
)

# Step 2: Aggregate total rides and cancelled rides
agg_df = (valid_trips
          .groupBy("request_dt")
          .agg(
              count("*").alias("total_rides"),
              _sum("is_cancelled").alias("cancelled_rides")
          ))

# Step 3: Compute cancellation_rate = total / cancelled
result = agg_df.withColumn(
    "cancellation_rate",
    round((col("total_rides") * 1.0) / col("cancelled_rides"), 2)
).orderBy("request_dt")

result.show()